# QuDiPy tutorial: loading control pulse (.ctrlp) and quantum circuit (.qcirc) files

This tutorial will walk you through how to load .ctrlp and .qcirc files and how to use respective objects that are generated after loading the files.


## 1. First load the relevant modules

In [ ]:
# Since we don't actually have a package yet for people to install.. 
# We need to add our folder to the PYTHONPATH in order for import to find qudipy
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import qudipy.circuit as circ

## 2. Loading control pulse files
Now we will go through how to load a bunch of control pulse files and discuss the `ControlPulse` class. 

In [ ]:
# Get the directory where .ctrlp files are loaded
pulse_dir = os.getcwd()+"/QuDiPy tutorial data/Control pulses/"

# Now make a list of all the .ctrlp files you want to load (these must include the FULL file path)
ctrl_files = ["ROTX_1_2.ctrlp", "CTRLZ_3_4.ctrlp", "H_1_2_4.ctrlp",
                  "ROTX_3.ctrlp", "CTRLX_2_3.ctrlp", "SWAP_1_2.ctrlp"]
pulse_files = []
for p in ctrl_files:
    pulse_files.append(pulse_dir + p)

Before we actually load the .ctrlp files, let's see what the contents of the .ctrlp file should look like.

In [ ]:
with open(pulse_files[1], 'r') as f:
    print(f.read())

Here we are looking at the `CTRLZ_3_4.ctrlp` file. There are three lines at the start of the file to describe the control pulse.  
- The first line, "Ideal gate:" specifies what is the equivalent ideal gate being implemetned by the control pulse.  This is used to simulate the ideal circuit later when dealing with QuantumCircuit objects.
- The second line, "Pulse type:" sepcifies whether the control pulse is described using experimental parameters (such as gate voltages and magnetic fields) or effective parameters (such as g, J, and things directly implemented in the effective Hamiltonian).
- The third line, "Pulse length:" specifies the length of the pulse in ps.

After those are specified, "Control pulses:" indicates the actual control pulse is going to be given. The next line contains the name of each control variables (V1, V2, ...). The next lines specify the acutal values for the control pulse. The control pulse will be linearly interpolated during the actual time simulation using the control pulse.  The first three lines of the .ctrlp file can appear in any order as long as "Control pulses:" is the last portion of the pulse file. 

In [ ]:
# Now load all the control pulse files using the load_pulses method
pulse_dict = circ.load_pulses(pulse_files) 

Let's check the returned dictionary from load_pulses and see what each `ControlPulse` object looks like.

In [ ]:
pulse_dict

As you can see, load_pulses returns a dictionary of `ControlPulse` objects. Each `ControlPulse` contains the same information that was specified in the .ctrlp file.

In [ ]:
print("Name:",pulse_dict['CTRLZ_3_4'].name)
print("Pulse type:",pulse_dict['CTRLZ_3_4'].pulse_type)
print("Length:",pulse_dict['CTRLZ_3_4'].length)
print("Ideal gate:",pulse_dict['CTRLZ_3_4'].ideal_gate)
print("Number of control variables:",pulse_dict['CTRLZ_3_4'].n_ctrls)
print("Control variable names:",pulse_dict['CTRLZ_3_4'].ctrl_names)
print("Control pulse values:",pulse_dict['CTRLZ_3_4'].ctrl_pulses)

After all of the pulse files we need are loaded, we can look at loading .qcirc (quanum circuit) files which will be the groundwork for doing actual simulations with the spin simulator.  First let's look at an example .qcirc file to understand the syntax.

In [ ]:
circ_dir = os.getcwd()+"/QuDiPy tutorial data/Quantum circuits/"
circ_file1 = circ_dir + "test_circuit1.qcirc"

with open(circ_file1, 'r') as f:
    print(f.read())

The first line of the .qcirc file denotes the number of qubits in the quantum circuit to be simulated. Right after that, we begin defining the circuit. 

The first element in a line denotes the name of the control pulse to apply during the simulation (i.e. `pulse_dict['CTRLZ_3_4'].name`). 

The next elements denote which qubits the ideal gate equivalent of the control pulse acts on. These are used to do simulations of the ideal quantum circuit to compare to the simulated circuit using control pulses. If every control pulse in the circuit has `pulse_dict['CTRLZ_3_4'].ideal_gate` specified, then an ideal quantum circuit can be simulated, otherwise it is not possible.  For single qubit gates (i.e. `ROTX_1_2`), any number of affected qubits can be supplied and the ideal simulator will broadcast that ideal quantum gate to all of those qubits.  For double qubit gates (i.e. `CTRLZ_3_4`), there must be only 2 qubits specified. The first qubit given will be assumed to be the control qubit (if applicable) and the second will be the target qubit. 

Let's go ahead and load the .qcirc file to make a `QuantumCircuit` object.

In [ ]:
# To load the .qcirc file we must pass both the full file name as well as a dictionary of ControlPulse objects
circ1 = circ.load_circuit(circ_file1, pulse_dict)
print(circ1)

So what properties does a `QuantumCircuit` have?

In [ ]:
print("Name:",circ1.name)
print("Number of qubits:",circ1.n_qubits)
print("Loaded pulses:",circ1.gates)
print("Ideal circuit:",circ1.ideal_circuit)
print("Circuit sequence:",circ1.circuit_sequence)

The most important class variable for the `QuantumCircuit` object is the `.circuit_sequence` variable. This variable is a list containing every gate in the sequence in order as specified in the .qcirc file. Each element in the list contains the gate's name, ideal gate equivalent, and qubits affected. We can access the next gate in the circuit sequence by calling `get_next_gate`.

In [ ]:
print(circ1.get_next_gate())
print(circ1.get_next_gate())
# This method will reset the tracker back to the start of the circuit sequence.
circ1.reset_circuit_index()

In case you want to check the .qcirc file you loaded, there is a class method that allows you to see the equivalent ideal circuit for the pulse file you loaded.  The ideal circuit shown here will be the one that is also simulated in tandem with the control pulse simulation.

In [ ]:
circ1.print_ideal_circuit()

Now, the above tutorial has been assuming that you are dealing with simulating circuit using control pulses. However, you may not want to do that and instead only simulate a simple quantum circuit like the one displayed above. For a situation like that, how should you do the .qcirc file? Let's look at such an example.

In [ ]:
circ_dir = os.getcwd()+"/QuDiPy tutorial data/Quantum circuits/"
circ_file2 = circ_dir + "test_circuit2.qcirc"

with open(circ_file2, 'r') as f:
    print(f.read())

This is another quantum circuit file composed of only ideal gates keywords.  If an ideal gate keyword is specified in the .qcirc file, then it will NOT use a control pulse and instead just the ideal gate specified.  You must be careful not to name any .ctrlp files using an ideal gate keyword as `QuDiPy` will assume it is an ideal gate instead.

For a list of available ideal gate keywords, we can check the following docstring...

In [ ]:
print(circ.check_ideal_gate.__doc__)

Now if we load the .qcirc circuit file, we don't need to supply a pulse dictionary because all the gates in the file are ideal.

In [ ]:
circ2 = circ.load_circuit(circ_file2)
circ2.print_ideal_circuit()